In [4]:
import tensorflow as tf
from datasets import load_dataset
from transformers import TFMT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq
from tensorflow.keras.optimizers import Adam


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 24.0 MB/s eta 0:00:00


In [5]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
from datasets import Dataset, DatasetDict
import pandas as pd
import csv

train_df = pd.read_csv("/content/translated_train.csv", sep=None, engine="python", on_bad_lines='skip')
test_df = pd.read_csv("/content/translated_test.csv", sep=None, engine="python", on_bad_lines='skip')



# Pandas DataFrame'leri Hugging Face Dataset formatına çevirme
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 📌 Eğitim veri setinin tamamını kullanmak istersen:
# combined_datasets = DatasetDict({
#     "train": train_dataset,
#     "test": test_dataset
# })

# Hugging Face DatasetDict oluşturma
combined_datasets = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})
# Mevcut sütun isimlerini kontrol et
print("Test dataset columns:", combined_datasets["test"].column_names)

# Sütun ismini değiştirmek için:
combined_datasets["test"] = combined_datasets["test"].rename_column("\ufeffarticle", "article")

# Tekrar kontrol et
print("Updated test dataset columns:", combined_datasets["test"].column_names)


# Sonuçları kontrol et
print(combined_datasets)

'''from statistics import mean

# Train datasetindeki article ve highlights sütunlarını al
articles = combined_datasets["train"]["article"]
highlights = combined_datasets["train"]["highlights"]

# Her bir metni tokenize edip uzunluklarını hesapla
article_lengths = [len(tokenizer.tokenize(text)) for text in articles]
highlight_lengths = [len(tokenizer.tokenize(text)) for text in highlights]

# Ortalama uzunluğu hesapla
avg_article_length = round(mean(article_lengths))
avg_highlight_length = round(mean(highlight_lengths))

print(f"📌 Ortalama Article Uzunluğu: {avg_article_length} token")
print(f"📌 Ortalama Highlights Uzunluğu: {avg_highlight_length} token")'''


Test dataset columns: ['\ufeffarticle', 'highlights']
Updated test dataset columns: ['article', 'highlights']
DatasetDict({
    train: Dataset({
        features: ['article', 'highlights'],
        num_rows: 16901
    })
    test: Dataset({
        features: ['article', 'highlights'],
        num_rows: 1400
    })
})


'from statistics import mean\n\n# Train datasetindeki article ve highlights sütunlarını al\narticles = combined_datasets["train"]["article"]\nhighlights = combined_datasets["train"]["highlights"]\n\n# Her bir metni tokenize edip uzunluklarını hesapla\narticle_lengths = [len(tokenizer.tokenize(text)) for text in articles]\nhighlight_lengths = [len(tokenizer.tokenize(text)) for text in highlights]\n\n# Ortalama uzunluğu hesapla\navg_article_length = round(mean(article_lengths))\navg_highlight_length = round(mean(highlight_lengths))\n\nprint(f"📌 Ortalama Article Uzunluğu: {avg_article_length} token")\nprint(f"📌 Ortalama Highlights Uzunluğu: {avg_highlight_length} token")'

In [19]:
def tokenize_sample_data(data):
    input_feature = tokenizer(
        data['article'],
        truncation=True,
        max_length=1150,
        padding="max_length"  # 🔥 Sabit uzunlukta padding ekle
    )
    label = tokenizer(
        data['highlights'],
        truncation=True,
        max_length=128,
        padding="max_length"
    )

    # 🔥 PAD token'larını -100 ile değiştir
    label_ids = label['input_ids']
    label_ids = [[-100 if token == tokenizer.pad_token_id else token for token in seq] for seq in label_ids]

    return {
        "input_ids": input_feature['input_ids'],
        "attention_mask": input_feature['attention_mask'],
        "labels": label_ids,  # 🔥 -100 ile düzeltilmiş labels
    }

tokenized_ds = combined_datasets.map(
    tokenize_sample_data,
    batched=True  # 🔥 batch_size belirtmene gerek yok
)


Map:   0%|          | 0/19519 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [ ]:
'''

from transformers import MT5Tokenizer, MT5ForConditionalGeneration

# 1. Tokenizer ve modelin yüklenmesi
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# 2. Özetlenmek istenen Türkçe metin
turkish_text = """
İklim değişikliği, küresel sıcaklıkların artışıyla birlikte ortaya çıkan çevresel, sosyal ve ekonomik sorunlara neden olmaktadır.
Özellikle kuraklık, sel ve orman yangınları gibi doğal afetlerin sıklığı artarken, tarımsal üretimde de ciddi düşüşler yaşanmaktadır.
Bu durum, gıda güvenliğini tehdit etmekte ve toplumları olumsuz yönde etkilemektedir.
"""

# 3. Özetleme için giriş formatı: 'summarize:' ön ekini ekliyoruz
input_text = f"summarize: {turkish_text}"

# 4. Tokenizer ile metni encode etme
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

# 5. Modeli kullanarak özetleme yapma
summary_ids = model.generate(
    input_ids,
    max_length=150,
    min_length=30,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)

# 6. Özetlenen metni decode etme
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 7. Sonucu yazdırma
print("Özetlenen Metin:")
print(summary)
'''

'\n\nfrom transformers import MT5Tokenizer, MT5ForConditionalGeneration\n\n# 1. Tokenizer ve modelin yüklenmesi\nmodel_name = "google/mt5-small"\ntokenizer = MT5Tokenizer.from_pretrained(model_name)\nmodel = MT5ForConditionalGeneration.from_pretrained(model_name)\n\n# 2. Özetlenmek istenen Türkçe metin\nturkish_text = """\nİklim değişikliği, küresel sıcaklıkların artışıyla birlikte ortaya çıkan çevresel, sosyal ve ekonomik sorunlara neden olmaktadır.\nÖzellikle kuraklık, sel ve orman yangınları gibi doğal afetlerin sıklığı artarken, tarımsal üretimde de ciddi düşüşler yaşanmaktadır.\nBu durum, gıda güvenliğini tehdit etmekte ve toplumları olumsuz yönde etkilemektedir.\n"""\n\n# 3. Özetleme için giriş formatı: \'summarize:\' ön ekini ekliyoruz\ninput_text = f"summarize: {turkish_text}"\n\n# 4. Tokenizer ile metni encode etme\ninput_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)\n\n# 5. Modeli kullanarak özetleme yapma\nsummary_ids = model.genera

In [21]:
import torch
from transformers import AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli yükle (önceden tanımladığın tokenizer'ı kullan)
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small").to(device)


In [22]:
data_collator = DataCollatorForSeq2Seq(
  tokenizer,
  model=model,
  return_tensors="pt")

In [23]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7d0f5b15d7ccf82babe5c3809fb2bb0b364730b654ee1095ebe5669eec0dc360
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [24]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
    return arg  # Bu fonksiyon kaldırılabilir veya değiştirilmeden kullanılabilir

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  # (Note : Please change this code, when you perform on other languages except for Japanese)
  text_preds = [(p if p.endswith((".", "!", "?")) else p + ".") for p in text_preds]
  text_labels = [(l if l.endswith((".", "!", "?")) else l + ".") for l in text_labels]

  # Türkçe cümle tokenizasyonu
  sent_tokenizer_tr = RegexpTokenizer(r'[^.!?]*[.!?]')
  text_preds = ["\n".join(np.char.strip(sent_tokenizer_tr.tokenize(p))) for p in text_preds]
  text_labels = ["\n".join(np.char.strip(sent_tokenizer_tr.tokenize(l))) for l in text_labels]

  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

In [3]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="mt5-summarize-tr",
    log_level="error",
    num_train_epochs=10,  # 15 yerine 10 (overfitting riskini azaltmak için)
    learning_rate=2e-5,  # 3e-5 fazla olabilir, 2e-5 daha stabil
    lr_scheduler_type="linear",
    warmup_steps=800,  # 1000 yerine 800 (hızlı stabilizasyon için)
    optim="adafactor",
    weight_decay=0.01,
    per_device_train_batch_size=8,  # 20 yerine 8 (Bellek tasarrufu, daha stabil eğitim)
    per_device_eval_batch_size=8,   # 20 yerine 8
    gradient_accumulation_steps=8,  # 20 yerine 8 (toplam efektif batch size: 64)
    evaluation_strategy="steps",
    eval_steps=200,  # 250 yerine 200 (erken durdurma için daha sık değerlendirme)
    predict_with_generate=True,
    generation_max_length=256,
    save_steps=1000,  # 2000 yerine 1000 (daha sık model kaydı)
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=3,  # En iyi 3 modeli sakla, disk kullanımını optimize et
    report_to="none",  # WandB veya TensorBoard istemiyorsan "none"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [42]:
from transformers import Seq2SeqTrainer, EarlyStoppingCallback

# EarlyStoppingCallback ile erken durdurma tanımlıyoruz
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=4,  # Art arda kaç kez validation loss'un düşmemesine izin verilecek
    early_stopping_threshold=0.0  # Minimum değişiklik miktarı (opsiyonel)
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=metrics_func,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    callbacks=[early_stopping]  # Erken durdurma callback'i buraya ekleniyor
)

trainer.train()


{'loss': 23.2619, 'grad_norm': 909.0204467773438, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.0204918032786885}
{'loss': 22.4731, 'grad_norm': 3476.05224609375, 'learning_rate': 2.0000000000000003e-06, 'epoch': 2.040983606557377}
{'loss': 20.5294, 'grad_norm': 547.5570068359375, 'learning_rate': 3e-06, 'epoch': 3.0614754098360657}
{'loss': 18.129, 'grad_norm': 594.2139282226562, 'learning_rate': 4.000000000000001e-06, 'epoch': 4.081967213114754}
{'train_runtime': 3224.1827, 'train_samples_per_second': 30.27, 'train_steps_per_second': 0.074, 'train_loss': 20.185885874430337, 'epoch': 4.901639344262295}


TrainOutput(global_step=240, training_loss=20.185885874430337, metrics={'train_runtime': 3224.1827, 'train_samples_per_second': 30.27, 'train_steps_per_second': 0.074, 'train_loss': 20.185885874430337, 'epoch': 4.901639344262295})

In [1]:
# Eğitilen modeli kaydet
output_dir = "./trained_model"  # Kaydedilecek dizin
trainer.save_model(output_dir)  # Modeli ve tokenizer'ı kaydeder

# Tokenizer'ı ayrıca kaydetmek isterseniz
tokenizer.save_pretrained(output_dir)

print(f"Model ve tokenizer {output_dir} dizinine kaydedildi.")

NameError: name 'trainer' is not defined

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Eğitilen modeli ve tokenizer'ı yükle
model = AutoModelForSeq2SeqLM.from_pretrained("./trained_model")
tokenizer = AutoTokenizer.from_pretrained("./trained_model")


In [ ]:
test_text = "Dünyamızda 1900’lü yıllardan günümüze kadar dil öğretiminde çeşitli metinler kullanılmıştır. Bunlar “ edebi metinler, üretilmiş metinler, özgün ve özel metinler ” başlıkları altında toplanmıştır. Metinlerin seçimi dil öğretim yaklaşım ve yöntemlerine göre değişmektedir. Her yaklaşım kendine özgü metin kullanmıştır. Geleneksel yaklaşımda dil bilgisi kuralları, atasözleri, edebiyat, genel kültür gibi konuların öğretimine ağırlık verildiğinden edebi metinler kullanılmıştır. Davranışçı yaklaşımda dil davranış olarak ele alınmış, tekrar, taklit ve ezberleme yoluyla öğretilmiştir. Bu yaklaşımda edebi metinler yerine üretilmiş metinler kullanılmıştır. Bilişsel yaklaşımda “dil iletişim aracıdır” görüşü yayılmış ve özgün metinler kullanılmaya başlanmıştır. "


In [ ]:
'''inputs = tokenizer(
    test_text,
    return_tensors="pt",  # PyTorch tensörleri olarak döner
    max_length=1024,      # Modelin giriş sınırı
    truncation=True       # Çok uzun metinleri keser
)
'''

In [ ]:
# Özetleme
output_ids = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=128,  # Özet uzunluğu sınırı
    num_beams=4,     # Beam search kullanımı
    no_repeat_ngram_size=2,  # Tekrar eden n-gramları engeller
    length_penalty=0.6       # Kısa özetlere öncelik
)

# Özet çözümleme
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Özet:", summary)


In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''